In [ ]:
import pandas as pd
from pathlib import Path
import re
from typing import Dict
import numpy as np

: 

In [ ]:
DATA_DIR = Path("/projectnb/cs505ws/projects/grammar_ninja/data/grammar/raw")

: 

In [ ]:
coedit = pd.read_parquet(DATA_DIR.joinpath("coedit", "train.parquet"))

In [ ]:
coedit.head()

,_id,task,src,tgt
0,1,gec,Remove all grammatical errors from this text: ...,"For example, countries with a lot of deserts c..."
1,2,gec,Improve the grammaticality: As the number of p...,"As the number of people grows, the need for a ..."
2,3,gec,Improve the grammaticality of this sentence: B...,Besides some technological determinists that a...
3,4,gec,Remove all grammatical errors from this text: ...,Safety is one of the crucial problems that man...
4,5,gec,Fix grammaticality in this sentence: On one ha...,"On the one hand, more and more viruses and hac..."


In [ ]:
coedit.task.unique()

array(['gec', 'neutralize', 'simplification', 'paraphrase', 'coherence',
       'clarity'], dtype=object)

In [ ]:
coedit.task.value_counts()

task
gec               19823
paraphrase        15370
simplification    11440
coherence         10616
neutralize        10570
clarity            1252
Name: count, dtype: int64

In [ ]:
coedit.src.apply(lambda s: len(s)).max()

884

In [ ]:
coedit.src.apply(lambda s: len(s)).mean()

: 

In [ ]:
prompts = coedit.src.str.extract(r"^(.*?):")

: 

In [ ]:
prompts.iloc[:, 0].unique()

: 

In [ ]:
coedit["prompts"] = coedit.src.str.extract(r"^(.*?):")

: 

In [ ]:
coedit["input_text"] = coedit.src.str.extract(r":(.+)")

: 

In [ ]:
coedit.rename({"tgt": "output_text"}, axis=1)

: 

In [ ]:
wi_locness = pd.read_parquet(DATA_DIR.joinpath("wi_locness", "validation.parquet"))

: 

In [ ]:
wi_locness.head()

: 

In [ ]:
wi_locness.loc[0, "edits"]

: 

In [ ]:
wi_locness.loc[0, "text"]

: 

In [ ]:
start = wi_locness.loc[0, "edits"]["start"]
end = wi_locness.loc[0, "edits"]["end"]
replacement_text = wi_locness.loc[0, "edits"]["text"]

: 

In [ ]:
replacement_text

: 

In [ ]:
wi_locness.loc[0, "text"][start[2] : end[2]]

: 

In [ ]:
replacement_text[2]

: 

In [ ]:
def apply_corrections(text: str, corrections: Dict[str, np.ndarray]):
    # The corrected text starts as the original text
    corrected_text = text
    offset = 0

    # Iterate over the corrections
    for start, end, correction in zip(
        corrections["start"], corrections["end"], corrections["text"]
    ):
        # Adjust start and end with the offset
        start += offset
        end += offset

        # Replace the text if correction exists
        if correction:
            corrected_text = corrected_text[:start] + correction + corrected_text[end:]
            # Update the offset
            offset += len(correction) - (end - start)
            
        else:
            corrected_text = corrected_text[:start] + corrected_text[end:]
            # Update the offset
            offset -= (end - start)

    return re.sub("\n+", "\n", corrected_text)

: 

In [ ]:
wi_locness["corrected_text"] = wi_locness.apply(lambda row: apply_corrections(row["text"], row["edits"]), axis=1)

: 

In [ ]:
wi_locness.corrected_text.head()

: 

In [ ]:
print(len(wi_locness.corrected_text[0].split("\n")[0]))

: 

In [ ]:
wi_locness.edits[0]

: 

In [ ]:
wi_locness.text.apply(lambda s: len(s)).max()

: 

In [ ]:
wi_locness.text.apply(lambda s: len(s)).mean()

: 

In [ ]:
wi_locness.corrected_text.apply(lambda s: len(s)).mean()

: 

In [ ]:
# split by newlines to match format of other coedit dataset

print(wi_locness.text[0].split("\n"))

print(wi_locness.corrected_text[0].split("\n"))

: 

In [ ]:
wi_locness.corrected_text[0].split("\n")

: 

In [ ]:
alternative_prompts = [
    "Correct the grammatical errors in the following sentence:",
    "Revise this text for proper grammar:",
    "Improve the grammatical structure of this sentence:",
    "Transform the words into grammatically correct English:",
    "Edit this sentence to eliminate grammatical mistakes:"
]

: 

In [ ]:
np.concatenate([wi_locness.corrected_text[i].split("\n") for i in range(wi_locness.shape[0])])

: 

In [ ]:
random_prompts = np.random.choice(alternative_prompts, size=wi_locness.shape[0], replace=True)

: 

: 